# Two quantum dots test

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
import kwant

import numpy as np
import scipy.linalg as la

import sympy
sympy.init_printing(print_builtin=False)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from scipy.sparse import csc_matrix

# Hamiltonian, parameters, system

In [ ]:
Ly = 6
L1, L2, L3 = 10, 10, 10
a = 1

hamiltonian = "A * (k_x**2 + k_y**2) + V0 * V(site) - V0"
potential = lambda site: np.heaviside(site.pos[0] - L1 - a/2, 0) * np.heaviside(L1 + L2 - site.pos[0] + a/2, 0)

## create total system

In [ ]:
from kwant.continuum import discretize

In [ ]:
transverse_shape = lambda site: (-Ly/2 - a/2 < site.pos[1] < Ly/2 + a/2)

shape1 = lambda site: (0 < site.pos[0] < L1 + a/2) and transverse_shape(site)
shape2 = lambda site: (L1 + a/2 < site.pos[0] < L1 + L2 + a/2) and transverse_shape(site)
shape3 = lambda site: (L1 + L2 + a/2 < site.pos[0] < L1+L2+L3 + a/2) and transverse_shape(site)


prim_vecs =  a * np.eye(2)
template = discretize(hamiltonian, grid_spacing=a)

syst = kwant.Builder()

shape = lambda site: shape1(site) or shape2(site) or shape3(site)
syst.fill(template, shape, (a/2, 0))
syst = syst.finalized()

N = len(syst.sites)
kwant.plot(syst, fig_size=(10, 4), show=False);
# plt.xlim(0, 30);

In [ ]:
N

## test subregion shape functions

In [ ]:
# syst = kwant.Builder()
# syst.fill(template, shape1, (a, 0))
# syst.fill(template, shape3, (L2+L1+a, 0))

# syst = syst.finalized()
# kwant.plot(syst, fig_size=(10, 4), show=False);
# plt.xlim(0, 30);


# syst = kwant.Builder()
# syst.fill(template, shape2, (L1+a, 0))
# syst = syst.finalized()

# kwant.plot(syst, fig_size=(10, 4), show=False);
# plt.xlim(0, 30);

In [ ]:
V = np.array([potential(s) for s in syst.sites])

fig = plt.figure(figsize=(10, 4))
ax = fig.gca()

kwant.plot(syst, ax=ax);
kwant.plotter.map(syst, V, ax=ax, colorbar=True);

In [ ]:
V0 = 0.5
kwant.plotter.map(syst, V0 * V - V0);

# diagonalise

In [ ]:
params = dict(A=1, V0=.5, V=potential)
mat = syst.hamiltonian_submatrix(params=params)
ev_exact, evec_exact = la.eigh(mat)

In [ ]:
print(len(ev_exact[ev_exact<0]))

plt.hlines(ev_exact, 0, 1);
plt.ylim(None, .5);

In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
%%output size=400
%%opts Raster (cmap='viridis' interpolation='sinc') [colorbar=True]

coords = np.array([s.pos for s in syst.sites])

plots = {}
for i in range(10):
    wf = np.abs(evec_exact[:, i])**2
    wf = kwant.plotter.mask_interpolate(coords, wf, oversampling=1)[0]
    img = hv.Raster(np.rot90(wf))
    plots[ev_exact[i]] = img
    
hv.HoloMap(plots, kdims='E')    

# Solve separate halfs of the system and use hoppings as perturbation

In [ ]:
# Be sure to choose a proper value for the middle of the system.
Lmid = 15 + a/2

PL = kwant.operator.Density(syst, where=lambda site: site.pos[0] < Lmid, sum=True).tocoo()
PR = kwant.operator.Density(syst, where=lambda site: site.pos[0] > Lmid, sum=True).tocoo()


assert la.norm(PL + PR - np.eye(N)) == 0


PL = csc_matrix(PL)[PL.getnnz(1) > 0]
PR = csc_matrix(PR)[PR.getnnz(1) > 0]

In [ ]:
HL = PL @ mat @ PL.T
HR = PR @ mat @ PR.T

In [ ]:
evL, evecL = la.eigh(HL)
evR, evecR = la.eigh(HR)

In [ ]:
evecL = PL.T @ evecL
evecR = PR.T @ evecR

In [ ]:
%%output size=400
%%opts Raster (cmap='viridis' interpolation='sinc') [colorbar=True]

coords = np.array([s.pos for s in syst.sites])

plots_L = {}
for i in range(10):
    wf = np.abs(evecL[:, i])**2
    wf = kwant.plotter.mask_interpolate(coords, wf, oversampling=1)[0]
    img = hv.Raster(np.rot90(wf))
    
    plots_L[i] = img
    
    
plots_R = {}
for i in range(10):
    wf = np.abs(evecR[:, i])**2
    wf = kwant.plotter.mask_interpolate(coords, wf, oversampling=1)[0]
    img = hv.Raster(np.rot90(wf))
    
    plots_R[i] = img    
    
    
hv.HoloMap(plots_L, kdims='E') + hv.HoloMap(plots_R, kdims='E')    

## Gather together  and calculate effective model

In [ ]:
H0 = PL.T @ HL @ PL + PR.T @ HR @ PR
Hprime = mat - H0

In [ ]:
evec = np.hstack([evecL[:, :2], evecR[:, :2], evecL[:, 2:], evecR[:, 2:]])
ev = np.hstack([evL[:2], evR[:2], evL[2:], evR[2:]])

In [ ]:
indices = range(4)

In [ ]:
# for simplicity lets import everything now
from codes.lowdin import *

In [ ]:
%%time

M0 = (evec[:, indices].T.conj() @ H0 @ evec[:, indices]).real
M1 = first_order({sympy.sympify(1): Hprime}, evec[:, indices])
M2 = second_order_explicit({sympy.sympify(1): Hprime}, ev, evec, indices)

## Effective model energies

In [ ]:
model = M0 + M1[1] + M2[1]
ev_effective = np.sort(la.eigvalsh(model))
ev_effective

## Difference between separate halfs and effective

In [ ]:
la.norm(ev_effective - ev[:4])

## Difference between exact full system and effective

In [ ]:
la.norm(ev_effective - ev_exact[:4])

# Look into the model

In [ ]:
sympy.sympify(np.round(M0, 4))

In [ ]:
sympy.sympify(np.round(M1[1], 4))

In [ ]:
sympy.sympify(np.round(M2[1], 4))